# <span style="color:blue"> B1. Import Packages <span>

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Data Analysis and Manipulation Tool
import pandas as pd

## Shows Progress of the Task
from tqdm.notebook import tqdm
tqdm.pandas()

## Date and Time Manipulation Tool
import datetime as dt

## Scientific Computing Tool
import numpy as np

## Visualization Tool
import seaborn as sns

#  <span style="color:blue">B2. Import Data <span>

In [ ]:
## Logon Data
df_log = pd.read_csv('./Data(Assignment).csv')

In [ ]:
df_log.head(5)

In [ ]:
df_log.dtypes

# <span style="color:blue">B3. Preprocessing</span>

In [ ]:
df_log['Time_Logon'] = pd.to_datetime(df_log['Time_Logon'])

In [ ]:
df_log.head()

In [ ]:
df_log.dtypes

# <span style="color:blue">B4. After-hour System Access</span>

<i><b> <font size="4">Let's first define a function that can determine whether a specific date is a business day or not.</font> </b></i>

In [ ]:
def check_bizday(date):
    """
    This function classify dates into business days and non-business days.
    """
    return bool(len(pd.bdate_range(date, date)))

<i><b> <font size="4">Now, we apply the function to our given data.</font> </b></i>

In [ ]:
df_log['bizday'] = df_log['Time_Logon'].progress_apply(check_bizday)

<i><b> <font size="4">Wow, that was super fast. </font> </b></i>

<i><b> <font size="4">Now, let's extract the hour when the employee accessed the system. </font> </b></i>

In [ ]:
df_log['hour'] =df_log['Time_Logon'].dt.hour

<i><b> <font size="4">We make two tables for each business day access and non-business day access. </font> </b></i>

In [ ]:
df_log_biz = df_log.loc[df_log['bizday']==True]
df_log_nonbiz = df_log.loc[df_log['bizday']==False]

<i><b> <font size="4">Also, we count the frequency of after-hour access. </font> </b></i>

In [ ]:
df_log_biz_hour = df_log_biz.groupby(['hour']).count()[['Time_Logon']].reset_index()
df_log_nonbiz_hour = df_log_nonbiz.groupby(['hour']).count()[['Time_Logon']].reset_index()

<i><b> <font size="4"> Let's visualize the result. </font> </b></i>

<i><font size="3"> When the employee accessed the system? </font></i>

In [ ]:
sns.barplot(x="hour", y="Time_Logon", data=df_log_biz_hour)

In [ ]:
sns.barplot(x="hour", y="Time_Logon", data=df_log_nonbiz_hour)

<i><font size="3"> How often do they make after-hour access? </font></i>

In [ ]:
user_count_biz = df_log_biz.groupby(['Employee_number']).count()[['Time_Logon']]
user_count_nonbiz = df_log_nonbiz.groupby(['Employee_number']).count()[['Time_Logon']]

In [ ]:
sns.countplot(x="Time_Logon",data=user_count_biz)

In [ ]:
sns.countplot(x="Time_Logon",data=user_count_nonbiz)

<i><b> <font size="4"> Let's look on the employees who access the system during the after-hour for the first time </font> </b></i>

In [ ]:
suspicious_list = user_count_biz.loc[user_count_biz['Time_Logon']==1].index

In [ ]:
pd.DataFrame(suspicious_list).head()

In [ ]:
suspicious_list

# <span style="color:blue"> B5. Device Access </span>

<i><b> <font size="4"> Deeper look on the suspicious employees. We check whether they have inserted external devices to transfer data. </font> </b></i>

In [ ]:
df_suspicious = df_log_biz.loc[df_log_biz['Employee_number'].isin(suspicious_list)].dropna().reset_index(drop=True)

In [ ]:
df_table = df_suspicious[['Employee_name','History_Web']]

In [ ]:
df_table['History_Web'] = [i.split(',')[0] for i in df_table['History_Web']]

In [ ]:
df_table